# Genetic algorithms
We define an abstract class *Chromosome*  which has four essential features:
* determine its own fitness
* create an instance with randomly selected genes (fill the first generation)
* implement cross-over (mix itself with another chromosome)
* mutate

In [1]:
exec('from __future__ import annotations')
from abc import ABC, abstractmethod
from copy import deepcopy
from enum import Enum
from heapq import nlargest
from random import choices, random, randrange, sample, shuffle
from statistics import mean
from typing import TypeVar, Tuple, Type, Generic, List, Callable

In [2]:
T = TypeVar('T', bound='Chromosome')  # for returning self

class Chromosome(ABC):
    """
    Base class for all chromosomes, all methods must be overridden.
    """
    @abstractmethod
    def fitness(self) -> float:
        ...
        
    @classmethod
    @abstractmethod
    def random_instance(cls: Type[T]) -> T:
        ...
        
    @abstractmethod
    def crossover(self: T, other: T) -> Tuple[T, T]:
        ...
        
    @abstractmethod
    def mutate(self) -> None:
        ...

Steps of a genetic algorithm:
1. create an initial population of random chromosomes for the first generation
2. **measure** the fitness of each chromosome, if any exceeds the threshold, return
3. **select** individuals to reproduce, with a higher probability of selecting those with higher fitness
4. **crossover** some of the selected chromosomes to create the next generation
5. **mutate** some of those chromosomes
6. return to step 2 unless the maximum number of generations has been reached, in this case return the chromosome with the highest fitness

In [8]:
C = TypeVar('C', bound=Chromosome)  # type of the chromosome

class GeneticAlgorithm(Generic[C]):
    SelectionType = Enum('SelectionType', 'ROULETTE TOURNAMENT')
    
    def __init__(
        self,
        initial_population: List[C],
        threshold: float,
        max_generations: int = 100,
        mutation_chance: float = 0.01,
        crossover_chance: float = 0.7,
        selection_type: SelectionType = SelectionType.TOURNAMENT
    ) -> None:
        self._population: List[C] = initial_population
        self._threshold: float = threshold
        self._max_generations: int = max_generations
        self._mutation_chance: float = mutation_chance
        self._crossover_chance: float = crossover_chance
        self._selection_type: GeneticAlgorithm.SelectionType = selection_type
            
        # type refers to the specific subclass of Chromosome that we are finding the fitness of
        self._fitness_key: Callable = type(self._population[0]).fitness
            
    def _pick_roulette(self, wheel: List[float]) -> Tuple[C, C]:
        """
        Use the probability distribution wheel to pick 2 parents.
        This will not work with negative fitness results.
        """
        return tuple(choices(self._population, weights=wheel, k=2))
    
    def _pick_tournament(self, num_participants: int) -> Tuple[C, C]:
        """
        Choose number of participants at random and take the best 2.
        Optimal number of participants is determined by trial and error.
        Higher number of participants leads to less diversity in population,
        because chromosomes with poor fitness are more likely to be eliminated
        in matchups.
        """
        participants: List[C] = choices(self._population, k=num_participants)
        return tuple(nlargest(2, participants, key=self._fitness_key))
    
    def _reproduce_and_replace(self) -> None:
        new_population: List[C] = []

        # keep going until we've filled the new generation
        while len(new_population) < len(self._population):
            # pick 2 parents
            if self._selection_type == GeneticAlgorithm.SelectionType.ROULETTE:
                parents: Tuple[C, C] = self._pick_roulette([x.fitness() for x in self._population])
            else:
                
                parents = self._pick_tournament(len(self._population) // 2)

            # potentially crossover the 2 parents
            if random() < self._crossover_chance:
                new_population.extend(parents[0].crossover(parents[1]))
            else:
                new_population.extend(parents)
            
        # if we had an odd number we'll have one extra so we remove it
        if len(new_population) > len(self._population):
            new_population.pop()
        
        self._population = new_population
        
    def _mutate(self) -> None:
        for individual in self._population:
            if random() < self._mutation_chance:
                individual.mutate()
                
    def run(self) -> C:
        best: C = max(self._population, key=self._fitness_key)
        for generation in range(self._max_generations):
            if best.fitness() >= self._threshold:
                return best
            print(f'Generation {generation} Best {best.fitness()} '
                  f'Average {mean(map(self._fitness_key, self._population))}')
            self._reproduce_and_replace()
            self._mutate()
            highest: C = max(self._population, key=self._fitness_key)
            if highest.fitness() > best.fitness():
                best = highest
        return best

Genetic algorithm selection methods include **roulette wheel** and **tournament selection**. Roulette wheel gives every chromosome a chance of being picked, proportionate to its fitness. In tournament selection, a certain number of chromosomes are challenged against one another and the one with the best fitness is selected.

In [4]:
class SimpleEquation(Chromosome):
    def __init__(self, x: int, y: int) -> None:
        self.x = x
        self.y = y
        
    def fitness(self) -> float:
        return 6 * self.x - self.x ** 2 + 4 * self.y - self.y ** 2
    
    @classmethod
    def random_instance(cls) -> C:
        return cls(randrange(100), randrange(100))
    
    def crossover(self, other) -> Tuple[C, C]:
        child1: C = deepcopy(self)
        child2: C = deepcopy(other)
        child1.y = other.y
        child2.y = self.y
        return child1, child2
    
    def mutate(self) -> None:
        if random() > 0.5:
            if random() > 0.5:
                self.x += 1
            else:
                self.x -= 1
        else:
            if random() > 0.5:
                self.y += 1
            else:
                self.y -= 1
                
    def __str__(self) -> str:
        return f'X: {self.x} Y: {self.y} Fitness: {self.fitness()}'

In [5]:
initial_population: List[SimpleEquation] = [SimpleEquation.random_instance() for _ in range(20)]
ga: GeneticAlgorithm[SimpleEquation] = GeneticAlgorithm(
    initial_population=initial_population,
    threshold=13,
    max_generations=100,
    mutation_chance=0.1,
    crossover_chance=0.7)
result: SimpleEquation = ga.run()
print(result)

Generation 0 Best -428 Average -6558.6
X: 3 Y: 2 Fitness: 13


# Revisiting SEND MORE MONEY

In [14]:
class SendMoreMoney2(Chromosome):
    def __init__(self, letters: List[str]) -> None:
        self.letters: List[str] = letters
            
    def fitness(self) -> float:
        s: int = self.letters.index('S')
        e: int = self.letters.index('E')
        n: int = self.letters.index('N')
        d: int = self.letters.index('D')
        m: int = self.letters.index('M')
        o: int = self.letters.index('O')
        r: int = self.letters.index('R')
        y: int = self.letters.index('Y')
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m* 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - send - more)
        return 1 / (difference + 1)
    
    @classmethod
    def random_instance(cls) -> C:
        letters = ['S', 'E', 'N', 'D', 'M', 'O', 'R', 'Y', ' ', ' ']
        shuffle(letters)
        return cls(letters)
    
    def crossover(self, other) -> Tuple[C, C]:
        child1: C = deepcopy(self)
        child2: C = deepcopy(other)
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        l1, l2 = child1.letters[idx1], child2.letters[idx2]
        child1.letters[child1.letters.index(l2)], child1.letters[idx2] = child1.letters[idx2], l2
        child2.letters[child2.letters.index(l1)], child2.letters[idx1] = child2.letters[idx1], l1
        return child1, child2
    
    def mutate(self) -> None:
        """Swap two letter locations."""
        idx1, idx2 = sample(range(len(self.letters)), k=2)
        self.letters[idx1], self.letters[idx2] = self.letters[idx2], self.letters[idx1]
        
    def __str__(self) -> str:
        s: int = self.letters.index('S')
        e: int = self.letters.index('E')
        n: int = self.letters.index('N')
        d: int = self.letters.index('D')
        m: int = self.letters.index('M')
        o: int = self.letters.index('O')
        r: int = self.letters.index('R')
        y: int = self.letters.index('Y')
        send: int = s * 1000 + e * 100 + n * 10 + d
        more: int = m * 1000 + o * 100 + r * 10 + e
        money: int = m* 10000 + o * 1000 + n * 100 + e * 10 + y
        difference: int = abs(money - send - more)
        return f'{send} + {more} = {money} Difference: {difference}'

In [17]:
initial_population: List[SendMoreMoney2] = [SendMoreMoney2.random_instance() for _ in range(1000)]
ga: GeneticAlgorithm[SendMoreMoney2] = GeneticAlgorithm(
    initial_population=initial_population,
    threshold=1.0,
    max_generations=1000,
    mutation_chance=0.2,
    crossover_chance=0.7,
    selection_type=GeneticAlgorithm.SelectionType.ROULETTE)
result: SendMoreMoney2 = ga.run()
print(result)

Generation 0 Best 0.009433962264150943 Average 0.0001411821764919521
Generation 1 Best 0.16666666666666666 Average 0.003452672069367096
6419 + 724 = 7143 Difference: 0


In [23]:
print(initial_population[0].letters)
print(initial_population[1].letters)
a, b = initial_population[0].crossover(initial_population[1])
print()
print(a.letters)
print(b.letters)

['M', 'O', 'Y', 'D', ' ', 'R', 'E', 'N', 'S', ' ']
['D', 'E', 'S', 'Y', 'N', ' ', 'R', 'M', ' ', 'O']

['M', 'O', 'D', 'Y', ' ', 'R', 'E', 'N', 'S', ' ']
['D', 'E', 'S', 'Y', 'M', ' ', 'R', 'N', ' ', 'O']
